In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
dfestu = {}
dfcole = {}
for i in coleDic:
    dfestu[i] = None
    dfcole[i] = None

In [7]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [8]:
print("Archivos de estudiantes: ", estuDic['20061'],estuDic['20062'])
print("Archivos de colegios: ",coleDic['2006'])

Archivos de estudiantes:  SB11_20061.txt SB11_20062.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2006.txt


In [9]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20061'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20062'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [10]:
columnasPuntos = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [11]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in columnasPuntos:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    
df1 = df2 = None

In [12]:
print("Numero de filas ", df.count())

Numero de filas  471878


In [13]:
print(df.printSchema())

root
 |-- estu_exam_nombreexamen: string (nullable = true)
 |-- estu_estudiante: integer (nullable = true)
 |-- periodo: integer (nullable = true)
 |-- estu_consecutivo: string (nullable = true)
 |-- estu_edad: integer (nullable = true)
 |-- estu_tipo_documento: string (nullable = true)
 |-- estu_pais_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_nacimiento_dia: integer (nullable = true)
 |-- estu_nacimiento_mes: integer (nullable = true)
 |-- estu_nacimiento_anno: integer (nullable = true)
 |-- estu_cod_reside_mcpio: integer (nullable = true)
 |-- estu_reside_mcpio: string (nullable = true)
 |-- estu_reside_depto: string (nullable = true)
 |-- estu_zona_reside: integer (nullable = true)
 |-- estu_razoninstituto: integer (nullable = true)
 |-- cole_valor_pension: integer (nullable = true)
 |-- estu_ies_cod_deseada: integer (nullable = true)
 |-- estu_carrdeseada_razon: integer (nullable = true)
 |-- estu_carrdeseada_cod: integer (nullable = true)

In [14]:
df.columns

['estu_exam_nombreexamen',
 'estu_estudiante',
 'periodo',
 'estu_consecutivo',
 'estu_edad',
 'estu_tipo_documento',
 'estu_pais_reside',
 'estu_genero',
 'estu_nacimiento_dia',
 'estu_nacimiento_mes',
 'estu_nacimiento_anno',
 'estu_cod_reside_mcpio',
 'estu_reside_mcpio',
 'estu_reside_depto',
 'estu_zona_reside',
 'estu_razoninstituto',
 'cole_valor_pension',
 'estu_ies_cod_deseada',
 'estu_carrdeseada_razon',
 'estu_carrdeseada_cod',
 'estu_ies_deseada_nombre',
 'estu_ies_cod_mpio_deseada',
 'estu_ies_mpio_deseada',
 'estu_ies_dept_deseada',
 'estu_mpio_presentacion',
 'estu_dept_presentacion',
 'estu_exam_cod_mpio_presentacio',
 'estu_ano_termino_bachill',
 'cole_cod_icfes',
 'cole_cod_dane_institucion',
 'cole_nombre_sede',
 'cole_cod_mcpio_ubicacion',
 'cole_calendario',
 'cole_genero',
 'cole_naturaleza',
 'cole_bilingue',
 'cole_jornada',
 'cole_caracter',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_f

In [15]:
# Seleccion de todas las instituciones diferentes
dsIns_dane = df.select('cole_cod_dane_institucion').distinct()

In [16]:
dsIns_dane.count()

7750

In [17]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=
False)

In [18]:
dsCountEst.show(10,False)

+-------------------------+-----+
|cole_cod_dane_institucion|count|
+-------------------------+-----+
|null                     |7531 |
|105001000108             |1200 |
|111001024732             |987  |
|376001001132             |843  |
|168001003591             |817  |
|111001019411             |811  |
|105001013340             |798  |
|113001003274             |702  |
|108758000031             |632  |
|305440000994             |627  |
+-------------------------+-----+
only showing top 10 rows



In [19]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_idioma']+df['punt_profundizacion']+df['punt_interdisciplinar'])

In [20]:
#Se crea una lista de columnas para hacer el analisis.
columnasToAnalysis = columnasPuntos.copy()
columnasToAnalysis.insert(0,'cole_cod_dane_institucion')
columnasToAnalysis.append('Suma')

In [21]:
columnasToAnalysis

['cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

In [22]:
# SE Selecciona el subgrupo de informciòn que se analizara
df = df.select(columnasToAnalysis)

In [23]:
df.columns

['cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

## Resultados finales por instituciòn

In [24]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('cole_cod_dane_institucion').mean()
df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df = df.orderBy('rank',ascending= True)

In [25]:
df.select('cole_cod_dane_institucion','avg(Suma)','rank').show(10,False)

+-------------------------+-----------------+----+
|cole_cod_dane_institucion|avg(Suma)        |rank|
+-------------------------+-----------------+----+
|311001089221             |590.5481338500977|1   |
|311001005176             |572.339729309082 |2   |
|311769003342             |568.1466674804688|3   |
|311848002611             |566.3491700490316|4   |
|354001000370             |565.3172607421875|5   |
|311001065489             |563.7452681692023|6   |
|311848002424             |563.3404284538107|7   |
|308001004209             |563.0185600359415|8   |
|305001015503             |562.9925079345703|9   |
|373001007662             |562.4288889567057|10  |
+-------------------------+-----------------+----+
only showing top 10 rows



## Colegios 2006

In [26]:
# se crea un dataframe con una columna que lo señala el año 2006.
from pyspark.sql.functions import lit
df = df.withColumn('año',lit(2006))
df.show(5,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica) |avg(punt_idioma) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|año |
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|311001089221             |65.57875061035156 |65.78999948501587    |69.42500019073486   |61.70125079154968  |69.47937536239624 |61.438124656677246|62.29437446594238|70.63875007629395|6.42750009894371        |

In [27]:
# SE crea Dataframe de colegios 2006
dfc = spark.read.load(base + '/' + coleDic['2006'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))
dfc = dfc.withColumn('COLE_CODMPIO_COLEGIO', dfc['COLE_CODMPIO_COLEGIO'].cast("string"))

In [28]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTE

In [29]:
dfc.columns

['COLE_CODIGO_COLEGIO',
 'COLE_INST_NOMBRE',
 'COLE_CODMPIO_COLEGIO',
 'COLE_MPIO_COLEGIO',
 'COLE_DEPTO_COLEGIO',
 'COLE_INST_JORNADA',
 'COLE_CALENDARIO_COLEGIO',
 'COLE_GENEROPOBLACION',
 'COLE_NATURALEZA',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA',
 'COLE_CATEGORIA',
 'COLE_ESTUDIANTES_PRESENTES']

In [30]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [31]:
dfc = dfc.select(colsColegio)

In [35]:
dfc.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|127696             |5001                |6                     |6           |7          |6            |8             |6               |7            |null       |null          |null         |
|127704             |20001               |7                     |7           |7          |7            |8             |6               |7            |null       |null          |null         |
|127712             |41807              

In [36]:
print("El numero de colegios año 2006 es: ", dfc.select('COLE_CODIGO_COLEGIO').distinct().count())

El numero de colegios año 2006 es:  9249


## Anàlisis comparativo entre estudiantes y colegios año 2006

In [38]:
dfjoin2006 = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [92]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [39]:
dfjoin2006 = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODMPIO_COLEGIO)
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.